# Estimating groundwater storage

In [1]:
%matplotlib widget

import __init__
import scripts.config as config
import numpy as np
import pandas as pd
import tempfile
import datetime
from sklearn.svm import SVR
import geopandas as gpd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from matplotlib.font_manager import FontProperties
import seaborn as sns
# import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
import HydroErr as he

In [2]:
# Plotting parameters

XSMALL_SIZE = 6
SMALL_SIZE = 7
MEDIUM_SIZE = 9
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the figure title
plt.rcParams['figure.dpi'] = 140

In [3]:
# Import and format observed data (2003-2007 runoff)

input_dir = config.velma_data
results_dir = config.data_path.parents[0] / 'results' / 'ellsworth_baseline_03_07_12'

runoff_path = input_dir / 'runoff' / 'ellsworth_Q_2003_2007_dummy.csv'
runoff_start = pd.to_datetime('01-01-2003')
runoff_end = pd.to_datetime('12-31-2007')
nse_start = pd.to_datetime('01-01-2004')
nse_end = pd.to_datetime('12-31-2007')

runoff_obs = pd.read_csv(runoff_path, names=['runoff_obs'])
runoff_obs.index = pd.date_range(runoff_start, runoff_end)
runoff_obs['doy'], runoff_obs['year'] = runoff_obs.index.dayofyear, runoff_obs.index.year
runoff_obs = runoff_obs[(runoff_obs.index >= nse_start) & (runoff_obs.index <= nse_end)]

flow_path = config.streamflow
quality = pd.read_csv(flow_path, usecols=['Date', 'Quality'], parse_dates=True, index_col=0)
quality = quality[(quality.index >= nse_start) & (quality.index <= nse_end)]

precip_path = input_dir / 'precip' / 'PRISM_gauge_avg_ppt_2003_2019.csv'
forcing_start = pd.to_datetime('01-01-2003')
forcing_end = pd.to_datetime('12-31-2019')                     
precip = pd.read_csv(precip_path, names=['precip'])
precip.index = pd.date_range(forcing_start, forcing_end)
precip['doy'], precip['year'] = precip.index.dayofyear, precip.index.year
precip = precip[(precip.index >= nse_start) & (precip.index <= nse_end)]

temp_path = input_dir / 'temp' / 'ellsworth_temp_2003_2019.csv'
temp = pd.read_csv(temp_path, names=['temp'])
temp.index = pd.date_range(forcing_start, forcing_end)
temp['doy'], temp['year'] = temp.index.dayofyear, temp.index.year
temp = temp[(temp.index >= nse_start) & (temp.index <= nse_end)]

# Import VELMA outputs
velma_results = pd.read_csv(results_dir / 'DailyResults.csv')

# Format datetime of results
jday_pad = velma_results['Day'].apply(lambda x: str(x).zfill(3))
str_year = velma_results['Year'].apply(lambda x: str(x))
velma_results['year_jday'] = str_year + jday_pad
velma_results.index = pd.to_datetime(velma_results['year_jday'], format='%Y%j')
velma_results = velma_results[(velma_results.index >= nse_start) & (velma_results.index <= nse_end)]

## Calculating runoff ratio

#### Removing quality flagged values

In [9]:
runoff_sim = velma_results['Runoff_All(mm/day)_Delineated_Average']

df = pd.concat([runoff_sim, runoff_obs, precip.loc[:, 'precip'], quality], axis=1)
df_edited_precip = df.drop(df[(df['Quality'] == 160) | (df['Quality'] == 161) | (df['Quality'] == 179) | (df['Quality'] == 254)].index)

obs_yearly = pd.pivot_table(df_edited_precip[['runoff_obs', 'doy', 'year']], 
                            index=['doy'], 
                            columns=['year'], 
                            values=['runoff_obs'])

precip_yearly_edited = pd.pivot_table(df_edited_precip[['precip', 'doy', 'year']], 
                            index=['doy'], 
                            columns=['year'], 
                            values=['precip'])

In [10]:
display('yearly runoff_ratios: ', 
        np.divide(obs_yearly.sum(skipna=True).tolist(), 
                  precip_yearly_edited.sum(skipna=True).tolist()))

'yearly runoff_ratios: '

array([0.98255467, 0.9183338 , 0.97763636, 1.05792431])

## Calculate groundwater storage
GW storage = precip - (runoff + evapotranspiration)

In [41]:
cig_data_dir = config.data_path / 'precip' / 'VIC_WRF_EllsworthCr'
cig_file = 'flux_46.40625_-123.90625'
cig_cols = ["YEAR","MONTH","DAY","HOUR","OUT_PREC","OUT_PET_SHORT",
            "OUT_SWE","OUT_EVAP","OUT_RUNOFF","OUT_BASEFLOW",
            "OUT_SOIL_MOIST0", "OUT_SOIL_MOIST1","OUT_SOIL_MOIST2"]
pnnl_hist = pd.read_csv(cig_data_dir / 'pnnl_historical' / 'sim0' / cig_file, 
                        names=cig_cols, delimiter='\t', parse_dates={'DATE': ['YEAR', 'MONTH', 'DAY']},
                       index_col=0)

# Get daily sums
pnnl_hist_sum = pnnl_hist.groupby(pnnl_hist.index).sum()

pnnl_hist_sum_04_07 = pnnl_hist_sum[(pnnl_hist_sum.index >= nse_start) & (pnnl_hist_sum.index <= nse_end)]

In [69]:
df = pd.concat([runoff_sim, runoff_obs, precip.loc[:, 'precip'], pnnl_hist_sum_04_07.loc[:, 'OUT_EVAP'],
                pnnl_hist_sum_04_07.loc[:, 'OUT_PREC'], quality], axis=1)
df_edited = df.drop(df[(df['Quality'] == 160) | (df['Quality'] == 161) | (df['Quality'] == 179) | (df['Quality'] == 254)].index)

df_edited['gw'] = df_edited['precip'] - (df_edited['runoff_obs'] + df['OUT_EVAP'])
df_edited['gw'].describe()

count    1307.000000
mean       -1.161710
std         7.228315
min       -33.185699
25%        -4.369046
50%        -2.601002
75%         0.432998
max        64.040287
Name: gw, dtype: float64

In [70]:
df_edited['doy'], df_edited['year'] = df_edited.index.dayofyear, df_edited.index.year
precip_yearly = pd.pivot_table(df_edited, index=['doy'], columns=['year'], values=['precip'])
gw_yearly = pd.pivot_table(df_edited, index=['doy'], columns=['year'], values=['gw'])
runoff_obs_yearly = pd.pivot_table(df_edited, index=['doy'], columns=['year'], values=['runoff_obs'])
et_yearly = pd.pivot_table(df_edited, index=['doy'], columns=['year'], values=['OUT_EVAP'])
precip2_yearly = pd.pivot_table(df_edited, index=['doy'], columns=['year'], values=['OUT_PREC'])

In [72]:
# Observed vs. simulated runoff
years = precip_yearly.columns.get_level_values(1)
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))
for col, year in enumerate(years):
#     precip_yearly.iloc[:, col].plot(ax=axes[col], label='Precipitation', linewidth=1)
    precip2_yearly.iloc[:, col].plot(ax=axes[col], label='Precipitation2', linewidth=1)
#     gw_yearly.iloc[:, col].plot(ax=axes[col], label='Groundwater storage', linewidth=1)
    runoff_obs_yearly.iloc[:, col].plot(ax=axes[col], label='Observed runoff', linewidth=1)
#     et_yearly.iloc[:, col].plot(ax=axes[col], label='Evapotranspiration', linewidth=1)
    axes[col].set_title(year)
    axes[col].set_ylim([0, 80])
axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1.3), fancybox=True, ncol=2)
axes[0].set_ylabel('mm/day')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
# Export CIG precipitation
pnnl_hist_sum_03_19 = pnnl_hist_sum[(pnnl_hist_sum.index >= pd.to_datetime('2003-01-01')) & 
                                    (pnnl_hist_sum.index <= pd.to_datetime('2019-12-31'))]
precip2 = pnnl_hist_sum_03_19.loc[:, 'OUT_PREC']
outfile = str(config.velma_data / 'precip' / 'ellsworth_VIC_WRF_precip_03_19.csv')
precip2.to_csv(outfile, header=False, index=False)

## Groundwater storage estimation using PML data

In [73]:
# pd.read_csv(config.data_path / 'precip' / 'ellsworth' / 'pml_et_2002_2017_daily.csv')